##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex Pipelines 用の単純な TFX パイプライン


<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/gcp/vertex_pipelines_simple.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>
</td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/gcp/vertex_pipelines_simple.ipynb">     <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/gcp/vertex_pipelines_simple.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
<td><a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps%253A%252F%252Fraw.githubusercontent.com%252Ftensorflow%252Ftfx%252Fmaster%252Fdocs%252Ftutorials%252Ftfx%252Fgcp%252Fvertex_pipelines_simple.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Google Cloud Vertex AI Workbench で実行</a></td>
</table></div>


ノートブックを使用したこのチュートリアルでは、単純な TFX パイプラインを作成し、Google Cloud Vertex Pipelines で実行します。このノートブックは、[単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)で構築した TFX パイプラインを基盤として使用しています。TFX に精通しておらず、そのチュートリアルにまだ目を通していない場合は、このノートブックを進める前に読むことをお勧めします。

Google Cloud Vertex Pipelines では、サーバーレスの方法で ML ワークフローをオーケストレートすることで、ML システムの自動化、監視、および管理を行えます。ML パイプラインは、TFX で Python を使って定義してから Google Cloud で実行できます。Vertex Pipelines についての詳細は、[Vertex Pipelines の入門](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)をご覧ください。

このノートブックは、[Google Colab](https://colab.research.google.com/notebooks/intro.ipynb) または [AI Platform Notebooks](https://cloud.google.com/ai-platform-notebooks) で実行することを意図しています。いずれも使用していない場合は、上記の「Google Colab で実行」をクリックしてください。

## セットアップ

このノートブックを実行する前に必ず以下の項目を準備してください。

- [Google Cloud Platform](http://cloud.google.com/) プロジェクト
- [Google Cloud Storage](https://cloud.google.com/storage) バケット。[バケットの作成ガイド](https://cloud.google.com/storage/docs/creating-buckets)をご覧ください。
- [Vertex AI and Cloud Storage API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,storage-component.googleapis.com) の有効化

以降の GCP プロジェクトの構成については、[Vertex ドキュメント](https://cloud.google.com/vertex-ai/docs/pipelines/configure-project)をご覧ください。

### Python パッケージをインストールする

TFX や KFP など、ML パイプラインのオーサリングと Vertex Pipelines へのジョブの送信に必要な Python パッケージをインストールします。

In [ ]:
# Use the latest version of pip.
!pip install --upgrade pip
!pip install --upgrade "tfx[kfp]<2"

#### ランタイムを再起動しましたか？

Google Colab を使用している場合は、上記のセルを初めて実行した後に、「ランタイムを再起動」ボタンをクリックするか、「ランタイム」&gt;「ランタイムの再起動...」メニューを使って、ランタイムを再起動する必要があります。これは、Colab がパッケージを読み込むために必要な作業です。

Colab を使用していない場合は、以下のセルを使ってランタイムを再起動できます。

In [ ]:
# docs_infra: no_execute
import sys
if not 'google.colab' in sys.modules:
  # Automatically restart kernel after installs
  import IPython
  app = IPython.Application.instance()
  app.kernel.do_shutdown(True)

### このノートブックについて Google にログインする

このノートブックを Colab で実行している場合は、お使いのユーザーアカウントで認証を行います。

In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

**AI Platform Notebooks を使用している場合**は、次のセクションを実行する前に、以下を実行して Google Cloud で認証を行います。

```sh
gcloud auth login
```

この認証は、**ターミナルウィンドウ**（**ファイル** &gt; **新規**）で行います。この作業は、ノートブックインスタンスごとに 1 回だけ必要です。

パッケージのバージョンを確認します。

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))

### 変数をセットアップする

以下のパイプラインをカスタマイズするために使用する変数をいくつかセットアップします。以下の情報が必要です。

- GCP プロジェクト ID。[プロジェクト ID を特定する](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects)をご覧ください。
- パイプラインを実行する GCP リージョン。Vertex Pipelines を利用できるリージョンについての詳細は、[Vertex AI のロケーションガイド](https://cloud.google.com/vertex-ai/docs/general/locations#feature-availability)をご覧ください。
- パイプライン出力を保存する Google Cloud Storage バケット。

**以下のセルに必要な値を入力してから実行します**。


In [ ]:
GOOGLE_CLOUD_PROJECT = ''     # <--- ENTER THIS
GOOGLE_CLOUD_REGION = ''      # <--- ENTER THIS
GCS_BUCKET_NAME = ''          # <--- ENTER THIS

if not (GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_REGION and GCS_BUCKET_NAME):
    from absl import logging
    logging.error('Please set all required parameters.')

`gcloud` を使用するプロジェクトに設定します。

In [ ]:
!gcloud config set project {GOOGLE_CLOUD_PROJECT}

In [ ]:
PIPELINE_NAME = 'penguin-vertex-pipelines'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for input data.
DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# This is the path where your model will be pushed for serving.
SERVING_MODEL_DIR = 'gs://{}/serving_model/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))

### サンプルデータを準備する

[単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)と同じ [Palmer Penguins データセット](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)を使用します。

このデータセットには、範囲 [0,1] を持つようにすでに正規化された数値特徴量が 4 つあります。ペンギンの `species`（種）を予測する分類モデルを構築します。

データセットのコピーを作成する必要があります。TFX ExampleGen はディレクトリから入力を読み取るため、ディレクトリを作成してデータセットをそれにコピーする必要があるためです。

In [ ]:
!gsutil cp gs://download.tensorflow.org/data/palmer_penguins/penguins_processed.csv {DATA_ROOT}/

CSV ファイルを見てみましょう。

In [ ]:
!gsutil cat {DATA_ROOT}/penguins_processed.csv | head

## パイプラインを作成する

TFX パイプラインは、Python API を使って定義されています。ここでは、CsvExampleGen、Trainer、Pusher という 3 つのコンポーネントで構成されるパイプラインを定義します。このパイプラインとモデルの定義は、[単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)とほぼ同じです。

唯一、[ML Metadata](https://www.tensorflow.org/tfx/guide/mlmd) データベースの場所を特定するために使用する `metadata_connection_config` を設定する必要がない点が異なります。これは、Vertex Pipelines uses ではマネージドメタデータサービスを使用するため、ユーザーがそれに注意したり、パラメータを指定したりする必要がないためです。

パイプラインを実際に定義する前にまず、Trainer コンポーネントのモデルコードを書く必要があります。

### モデルコードを記述する

[単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)と同じモデルコードを使用します。

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils


from tfx import v1 as tfx
from tfx_bsl.public import tfxio

from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _make_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _make_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

このモジュールファイルを、パイプラインコンポーネントからアクセスできる GCS にコピーします。モデルのトレーニングは GCP で行われるため、このモデル定義をアップロードする必要があります。

そうしない場合は、モジュールファイルを含むコンテナイメージをビルドし、そのイメージをパイプラインで実行することが可能です。

In [ ]:
!gsutil cp {_trainer_module_file} {MODULE_ROOT}/

### パイプライン定義を記述する

TFX パイプラインを作成する関数を定義します。

In [ ]:
# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple and
# slightly modified because we don't need `metadata_path` argument.

def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     ) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=components)

## Vertex Pipelines でパイプラインを実行する

[単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)では、ローカル環境で実行する `LocalDagRunner` を使用しました。TFX にはパイプラインを実行するためのオーケストレータが複数用意されています。このチュートリアルでは、Vertex Pipelines と Kubeflow V2 DAG ランナーを使用します。

ランナーは、パイプラインを実際に実行するように定義する必要があります。TFX API を使用して、パイプラインをパイプライン定義フォーマットにコンパイルします。

In [ ]:
# docs_infra: no_execute
import os

PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'

runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=PIPELINE_DEFINITION_FILE)
# Following function will write the pipeline definition to PIPELINE_DEFINITION_FILE.
_ = runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        serving_model_dir=SERVING_MODEL_DIR))

生成された定義ファイルは、kfp クライアントを使用して送信することができます。

In [ ]:
# docs_infra: no_execute
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
import logging
logging.getLogger().setLevel(logging.INFO)

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

上記の出力に含まれるリンクにアクセスするか、[Google Cloud Console](https://console.cloud.google.com/) で［Vertex AI］&gt;［パイプライン］にアクセスすると、進捗状況を確認できます。